# FPI Model

### This model compares ESPN FPI (Football Power Index) to Bovada odds and suggests bets

### Organization
1. Get up to date FPI from ESPN- DONE
2. Get up to date odds from Bovada - difficult, use espn chalk instead
3. Money line implied odds comparison
4. Point spread implied odds comparison

_______________________________________________________________________________________________________________________________

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# up to date FPI function
# returns pandas dataframe with current FPI table

def getFPI():
    url = 'https://www.espn.com/nfl/fpi'
    r = requests.get(url)
    html = r.text
    
    # ESPN splits the FPI table into two sides
    
    # put the left table (team names) into a pandas dataframe    
    soup = BeautifulSoup(html)
    table1 = soup.find('table', {"class": "Table Table--align-right Table--fixed Table--fixed-left"})
    rows = table1.find_all('tr')
    teams_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        teams_data.append([element for element in cols if element])   

    teams_df = pd.DataFrame(teams_data)
    
    # put the right side table (FPI and other stats) into a pandas dataframe
    table2 = soup.find('table', {"class": "Table Table--align-right"})
    rows = table2.find_all('tr')
    stats_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        stats_data.append([element for element in cols if element])   

    stats_df = pd.DataFrame(stats_data)
    
    # combine into one dataframe and update headings
    df = pd.merge(teams_df, stats_df, left_index=True, right_index=True)
    headers = {'0_x': 'TEAM', 
               '0_y': 'W-L', 
               1 : 'FPI', 
               2: 'RK', 
               3: 'TRND', 
               4: 'OFF', 
               5: 'DEF', 
               6: 'ST', 
               7: 'SOS', 
               8: 'REM_SOS', 
               9: 'AVG_WP'}
    df = df.rename(index=str,columns=headers)
    return df

In [3]:
def getESPNlines():
    url = 'https://www.espn.com/nfl/lines'
    dfs = pd.read_html(url)
    
    return dfs

In [4]:
dfs_in = getESPNlines()

In [5]:
line_dfs = dfs_in[:]
for i, df in enumerate(dfs_in):

    line_dfs[i] = df

## How to calculate implied odds (%) based on American Money Line
 
### Negative American Odds (favorite)
implied probability = negative odds / (negative odds + 100) * 100

### Positive American Odds (underdog)
implied probability = 100 / (positive odds + 100) * 100

In [6]:
# add implied odds column to dfs and compare
lines_list = line_dfs[:]
for i, df in enumerate(line_dfs):
    # sort: underdogs, favorites    
    df = df.sort_values('LINE', ascending=False)
    
    # add total and points for each team row
    total, spread = df.iloc[0]['LINE'], df.iloc[1]['LINE']
    df['TOTAL'] = [total, total]
    df['SPREAD'] = [-spread, spread]
    
    # drop unnecessary columns
    df = df.drop(['REC (ATS)', 'OPEN', 'LINE'], axis=1)
  
    # sort: underdog, favorite
    df = df.sort_values(['ML'], ascending=False)
    
    # calculate implied odds from ML
    for index, row in df.iterrows():
        if row['ML'] > 0:
            underdog = (100*100/(row['ML']+100))
        else:
            favorite = 100*abs(row['ML'])/(abs(row['ML'])+100)
    df['Implied_Odds'] = [underdog,favorite]
    
    # calculate ML edge (FPI - Implied Odds)
    df['FPI'] = df['FPI'].str.strip(' %')
    df['FPI'] = df['FPI'].astype(float)
    
    df['ML_Edge'] = df['FPI'] - df['Implied_Odds']
    df['U/F?'] = ['underdog', 'favorite']
    
    df.columns.values[0] = 'TEAM'
    
    lines_list[i] = df

In [7]:
# convert list of games to dataframe, organize columns, index by team
lines = pd.concat(lines_list)
lines = lines[['TEAM', 'ML', 'SPREAD', 'TOTAL', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
team_lines = lines.set_index('TEAM')

points_FPI = getFPI()
points_FPI = points_FPI.set_index('TEAM')
points_FPI['FPI'] = points_FPI['FPI'].astype(float)
points_FPI

,W-L,FPI,RK,TRND,OFF,DEF,ST,SOS,REM_SOS,AVG_WP
TEAM,,,,,,,,,,
Baltimore Ravens,2-0,9.2,1,--,4.8,4.1,0.3,25,31,1
Kansas City Chiefs,2-0,6.7,2,--,8.0,-1.2,-0.0,23,15,18
San Francisco 49ers,1-1,4.6,3,2,1.9,2.3,0.4,24,2,4
Seattle Seahawks,2-0,4.5,4,--,5.0,-0.5,-0.0,12,14,7
New Orleans Saints,1-1,4.2,5,2,4.0,-0.1,0.3,5,22,13
Los Angeles Rams,2-0,3.8,6,6,3.0,0.8,0.1,14,8,3
Green Bay Packers,2-0,3.7,7,3,5.8,-1.6,-0.5,27,25,6
New England Patriots,1-1,3.0,8,1,-0.3,3.0,0.3,16,5,12
Tampa Bay Buccaneers,1-1,2.8,9,3,-0.4,3.7,-0.5,19,19,16


# NOTE

ESPN SOS (strength of schedule) and AVG_WP (average in-game win probability) appear to be inversely related (intuitive), but discrepencies could lead to betting edge. Room for more analysis

### Money Line Bets Sorted by Best Edge
only displays ML bets with a positive edge

In [8]:
# take positive edge, round values, and organize
pos_edge = team_lines[team_lines.ML_Edge > 0]
pos_edge = pos_edge[['ML', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
pos_edge = pos_edge.sort_values('ML_Edge', ascending=False)
pos_edge.round(decimals=3)

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
New York Jets,425,34.0,19.048,14.952,underdog
Denver Broncos,225,41.9,30.769,11.131,underdog
Washington,270,36.3,27.027,9.273,underdog
Carolina Panthers,240,36.3,29.412,6.888,underdog
Green Bay Packers,150,46.4,40.000,6.400,underdog
Minnesota Vikings,120,51.5,45.455,6.045,underdog
Las Vegas Raiders,230,36.3,30.303,5.997,underdog
Chicago Bears,155,44.8,39.216,5.584,underdog
Cincinnati Bengals,190,39.3,34.483,4.817,underdog


### Point Spreads
ESPN FPI is defined as the expected point margin vs an average opponent
Assume an average opponent has an FPI of zero

FPI favorite expected point spread = Favorite FPI - Underdog FPI
FPI underdog expected point spread = -FPI favorite expected point spread

In [9]:
# loop over each matchup and calculate
spread_list = lines_list.copy()
for i, df in enumerate(lines_list):
    
    # drop unnecessary columns
    df = df.drop(['ML', 'FPI', 'TOTAL', 'Implied_Odds', 'ML_Edge'], axis=1)

    # index to teams
    df = df.set_index('TEAM')

    # add FPI, calculate FPI spread, calculate points edge
    df = pd.concat([df, points_FPI['FPI']], axis=1, join='inner')
    fpi_spread = df.iloc[0]['FPI'] - df.iloc[1]['FPI']
    df['FPI_spread'] = [-fpi_spread, fpi_spread]
    df['Points_Edge'] = df['SPREAD'] - df['FPI_spread']
     
    spread_list[i] = df

In [10]:
# convert list to a dataframe, organize, take positive edge
spreads = pd.concat(spread_list)
spreads = spreads[['SPREAD', 'FPI', 'FPI_spread', 'Points_Edge', 'U/F?']]

points_edge = spreads[spreads.Points_Edge > 0]
points_edge = points_edge.sort_values('Points_Edge', ascending=False)
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
TEAM,,,,,
New York Jets,10.5,-4.3,4.1,6.4,underdog
San Francisco 49ers,-4.0,4.6,-9.9,5.9,favorite
Washington,7.0,-4.9,2.7,4.3,underdog
Las Vegas Raiders,6.0,0.5,2.5,3.5,underdog
Miami Dolphins,3.0,-4.5,-0.3,3.3,underdog
Los Angeles Rams,2.0,3.8,-1.0,3.0,underdog
Chicago Bears,3.0,-1.4,0.4,2.6,underdog
Cincinnati Bengals,5.5,-5.3,2.9,2.6,underdog
Green Bay Packers,3.0,3.7,0.5,2.5,underdog


__________________________________________________________

# Final Results

## Money Line Edge

In [11]:
pos_edge

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
New York Jets,425,34.0,19.047619,14.952381,underdog
Denver Broncos,225,41.9,30.769231,11.130769,underdog
Washington,270,36.3,27.027027,9.272973,underdog
Carolina Panthers,240,36.3,29.411765,6.888235,underdog
Green Bay Packers,150,46.4,40.000000,6.400000,underdog
Minnesota Vikings,120,51.5,45.454545,6.045455,underdog
Las Vegas Raiders,230,36.3,30.303030,5.996970,underdog
Chicago Bears,155,44.8,39.215686,5.584314,underdog
Cincinnati Bengals,190,39.3,34.482759,4.817241,underdog


## Points Edge

In [12]:
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
TEAM,,,,,
New York Jets,10.5,-4.3,4.1,6.4,underdog
San Francisco 49ers,-4.0,4.6,-9.9,5.9,favorite
Washington,7.0,-4.9,2.7,4.3,underdog
Las Vegas Raiders,6.0,0.5,2.5,3.5,underdog
Miami Dolphins,3.0,-4.5,-0.3,3.3,underdog
Los Angeles Rams,2.0,3.8,-1.0,3.0,underdog
Chicago Bears,3.0,-1.4,0.4,2.6,underdog
Cincinnati Bengals,5.5,-5.3,2.9,2.6,underdog
Green Bay Packers,3.0,3.7,0.5,2.5,underdog


___________________________________________________________

# Stuff to Add

* add opponent, home/away, record, weather to dataframes
* add strength of schedule to date anaylsis